In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0cdd97df0d380fb3eabb1324a31f35cb64ccc62ce950566f0602243c2fbea036
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier, RandomForestClassifier, LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import matplotlib.pyplot as plt

In [ ]:
# class MultiClassClassification:
#     def __init__(self, input_path):
#         self.spark = SparkSession.builder.appName("MultiClassClassification").getOrCreate()
#         self.data = self.spark.read.csv(input_path, header=False, inferSchema=True)
#         self.data = self.data.withColumnRenamed("_c0", "label")  # Rename the first column to 'label'
#         for i in range(1, 785):
#             self.data = self.data.withColumnRenamed(f"_c{i}", f"pixel{i-1}")
#         self.train_data, self.test_data = None, None

#     def preprocess_data(self):
#         feature_cols = [f"pixel{i}" for i in range(784)]
#         assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
#         assembled_data = assembler.transform(self.data).select("label", "features")
#         self.train_data, self.test_data = assembled_data.randomSplit([0.8, 0.2], seed=1234)

#     def train_evaluate(self, model):
#         model = model.fit(self.train_data)
#         predictions_train = model.transform(self.train_data)
#         predictions_test = model.transform(self.test_data)

#         evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
#         accuracy_train = evaluator.evaluate(predictions_train)
#         accuracy_test = evaluator.evaluate(predictions_test)

#         return accuracy_train, accuracy_test

#     def run_classifiers(self):
#         results = {}

#         # Multi-layer Perceptron
#         layers = [784, 128, 64, 10]
#         mlp = MultilayerPerceptronClassifier(layers=layers, labelCol="label", featuresCol="features", maxIter=100)
#         results['MLP'] = self.train_evaluate(mlp)

#         # Random Forest
#         rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100)
#         results['RandomForest'] = self.train_evaluate(rf)

#         # Linear SVM
#         lsvc = LinearSVC(labelCol="label", featuresCol="features", maxIter=100)
#         results['LinearSVM'] = self.train_evaluate(lsvc)

#         return results

#     def plot_results(self, results):
#         models = list(results.keys())
#         accuracies_train = [results[model][0] for model in models]
#         accuracies_test = [results[model][1] for model in models]

#         x = range(len(models))

#         fig, ax = plt.subplots()
#         bar_width = 0.35

#         bars1 = ax.bar(x, accuracies_train, bar_width, label='Train Accuracy')
#         bars2 = ax.bar([p + bar_width for p in x], accuracies_test, bar_width, label='Test Accuracy')

#         ax.set_xlabel('Model')
#         ax.set_ylabel('Accuracy')
#         ax.set_title('Accuracy by Model and Dataset')
#         ax.set_xticks([p + bar_width / 2 for p in x])
#         ax.set_xticklabels(models)
#         ax.legend()

#         plt.show()

In [4]:
class MultiClassClassification:
    def __init__(self, input_path):
        self.spark = SparkSession.builder \
            .appName("MultiClassClassification") \
            .config("spark.driver.memory", "4g") \
            .config("spark.executor.memory", "4g") \
            .getOrCreate()
        self.data = self.spark.read.csv(input_path, header=False, inferSchema=True)
        self.rename_columns()
        self.train_data, self.test_data = None, None

    def rename_columns(self):
        # Efficiently rename columns
        new_column_names = ["label"] + [f"pixel{i}" for i in range(784)]
        self.data = self.data.toDF(*new_column_names)

    def preprocess_data(self):
        feature_cols = [f"pixel{i}" for i in range(784)]
        assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
        assembled_data = assembler.transform(self.data).select("label", "features")
        self.train_data, self.test_data = assembled_data.randomSplit([0.8, 0.2], seed=1234)

    def train_evaluate(self, model):
        model = model.fit(self.train_data)
        predictions_train = model.transform(self.train_data)
        predictions_test = model.transform(self.test_data)

        evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
        accuracy_train = evaluator.evaluate(predictions_train)
        accuracy_test = evaluator.evaluate(predictions_test)

        return accuracy_train, accuracy_test

    def run_classifiers(self):
        results = {}

        # Multi-layer Perceptron
        layers = [784, 128, 64, 10]
        mlp = MultilayerPerceptronClassifier(layers=layers, labelCol="label", featuresCol="features", maxIter=100)
        results['MLP'] = self.train_evaluate(mlp)

        # Random Forest
        rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100)
        results['RandomForest'] = self.train_evaluate(rf)

        # One-vs-Rest with Linear SVM
        lsvc = LinearSVC(labelCol="label", featuresCol="features", maxIter=100)
        ovr = OneVsRest(classifier=lsvc)
        results['OneVsRest_LinearSVM'] = self.train_evaluate(ovr)

        return results

    def plot_results(self, results):
        models = list(results.keys())
        accuracies_train = [results[model][0] for model in models]
        accuracies_test = [results[model][1] for model in models]

        x = range(len(models))

        fig, ax = plt.subplots()
        bar_width = 0.35

        bars1 = ax.bar(x, accuracies_train, bar_width, label='Train Accuracy')
        bars2 = ax.bar([p + bar_width for p in x], accuracies_test, bar_width, label='Test Accuracy')

        ax.set_xlabel('Model')
        ax.set_ylabel('Accuracy')
        ax.set_title('Accuracy by Model and Dataset')
        ax.set_xticks([p + bar_width / 2 for p in x])
        ax.set_xticklabels(models)
        ax.legend(loc='lower right')

        plt.show()

In [5]:
classification = MultiClassClassification("/content/drive/MyDrive/Colab_Notebooks/mnist_mini.csv")
classification.preprocess_data()
results = classification.run_classifiers()
classification.plot_results(results)